In [1]:
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

#Spacy
import spacy
nlp = spacy.load('en')

# Other
import re
import json
import string
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /home/isuri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
#load data
import pandas as pd
reviews_train = pd.read_csv("hotel.csv").astype(str)

#show first 5 records
reviews_train.head()

,aspect,aspect_category,sentiment,review
0,place,RESTAURANT#GENERAL,negative,\nJudging from previous posts this used to be ...
1,staff,SERVICE#GENERAL,negative,"\nWe, there were four of us, arrived at noon -..."
2,nan,SERVICE#GENERAL,negative,"\nThey never brought us complimentary noodles,..."
3,food,FOOD#QUALITY,negative,\nThe food was lousy - too sweet or too salty ...
4,portions,FOOD#STYLE_OPTIONS,negative,\nThe food was lousy - too sweet or too salty ...


In [3]:
# reviews_train.columns
print(reviews_train.groupby('aspect_category').size().sort_values(ascending=False))

#how many categories
print("number of categories",reviews_train.aspect_category.nunique())

aspect_category
FOOD#QUALITY                849
SERVICE#GENERAL             449
RESTAURANT#GENERAL          422
AMBIENCE#GENERAL            255
FOOD#STYLE_OPTIONS          137
RESTAURANT#MISCELLANEOUS     98
FOOD#PRICES                  90
RESTAURANT#PRICES            80
DRINKS#QUALITY               47
DRINKS#STYLE_OPTIONS         32
LOCATION#GENERAL             28
DRINKS#PRICES                20
dtype: int64
number of categories 12


In [4]:
#create a word embedding of reviews data
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.review)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.review))

In [5]:
absa_model = Sequential()
absa_model.add(Dense(512, input_shape=(6000,), activation='relu'))
absa_model.add((Dense(256, activation='relu')))
absa_model.add((Dense(128, activation='relu')))
absa_model.add(Dense(12, activation='softmax'))
#compile model
absa_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [6]:
label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(reviews_train.aspect_category)
encoded_y = to_categorical(integer_category)

In [7]:
# fit aspect classifier
absa_model.fit(reviews_tokenized, encoded_y, epochs=100, verbose=1)

Epoch 1/100
2507/2507 [==============================] - 2s 866us/step - loss: 1.7995 - accuracy: 0.4360
Epoch 2/100
2507/2507 [==============================] - 2s 838us/step - loss: 1.1558 - accuracy: 0.6191
Epoch 3/100
2507/2507 [==============================] - 2s 839us/step - loss: 0.8637 - accuracy: 0.6737
Epoch 4/100
2507/2507 [==============================] - 2s 825us/step - loss: 0.6886 - accuracy: 0.6964
Epoch 5/100
2507/2507 [==============================] - 2s 825us/step - loss: 0.5957 - accuracy: 0.7140
Epoch 6/100
2507/2507 [==============================] - 2s 847us/step - loss: 0.5396 - accuracy: 0.7248
Epoch 7/100
2507/2507 [==============================] - 2s 833us/step - loss: 0.5133 - accuracy: 0.7204
Epoch 8/100
2507/2507 [==============================] - 2s 833us/step - loss: 0.4770 - accuracy: 0.7347
Epoch 9/100
2507/2507 [==============================] - 2s 845us/step - loss: 0.4570 - accuracy: 0.7371
Epoch 10/100
2507/2507 [==============================]

2507/2507 [==============================] - 2s 853us/step - loss: 0.3802 - accuracy: 0.7559
Epoch 79/100
2507/2507 [==============================] - 2s 822us/step - loss: 0.3703 - accuracy: 0.7599
Epoch 80/100
2507/2507 [==============================] - 2s 833us/step - loss: 0.3703 - accuracy: 0.7587
Epoch 81/100
2507/2507 [==============================] - 2s 827us/step - loss: 0.3701 - accuracy: 0.7507
Epoch 82/100
2507/2507 [==============================] - 2s 835us/step - loss: 0.3718 - accuracy: 0.7451
Epoch 83/100
2507/2507 [==============================] - 2s 826us/step - loss: 0.3693 - accuracy: 0.7583
Epoch 84/100
2507/2507 [==============================] - 2s 835us/step - loss: 0.3683 - accuracy: 0.7527
Epoch 85/100
2507/2507 [==============================] - 2s 830us/step - loss: 0.3690 - accuracy: 0.7535
Epoch 86/100
2507/2507 [==============================] - 2s 824us/step - loss: 0.3669 - accuracy: 0.7675
Epoch 87/100
2507/2507 [==============================] - 2

In [8]:
test_reviews = [
    "Good, fast service.",
    "The hostess was very pleasant.",
    "The bread was stale, the salad was overpriced and empty.",
    "The food we ordered was excellent, although I wouldn't say the margaritas were anything to write home about.",
    "This place has totally weird decor, stairs going up with mirrored walls - I am surprised how no one yet broke their head or fall off the stairs"
]

# Aspect preprocessing
# test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))
test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))
                             

# Models output
test_aspect_categories = label_encoder.inverse_transform(absa_model.predict_classes(test_aspect_terms))



In [9]:
for i,j in zip(test_reviews,test_aspect_categories):
    print("Review:"+" "+str(i)+" "+"is opinion about"+ " " +str(j))

Review: Good, fast service. is opinion about SERVICE#GENERAL
Review: The hostess was very pleasant. is opinion about SERVICE#GENERAL
Review: The bread was stale, the salad was overpriced and empty. is opinion about FOOD#STYLE_OPTIONS
Review: The food we ordered was excellent, although I wouldn't say the margaritas were anything to write home about. is opinion about FOOD#QUALITY
Review: This place has totally weird decor, stairs going up with mirrored walls - I am surprised how no one yet broke their head or fall off the stairs is opinion about AMBIENCE#GENERAL


In [10]:
#model architecture
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add((Dense(256, activation='relu')))
sentiment_model.add((Dense(128, activation='relu')))
sentiment_model.add(Dense(3, activation='softmax'))
#compile model
sentiment_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#encode the label variable
label_encoder = LabelEncoder()
integer_sentiment1 = label_encoder.fit_transform(reviews_train.sentiment)
encoded_y1 = to_categorical(integer_sentiment1)
dummy_new = pd.get_dummies(integer_sentiment1)

In [11]:

#fit sentiment classifier
sentiment_model.fit(reviews_tokenized, dummy_new, epochs=100, verbose=1)

Epoch 1/100
2507/2507 [==============================] - 2s 868us/step - loss: 0.6334 - accuracy: 0.7419
Epoch 2/100
2507/2507 [==============================] - 2s 825us/step - loss: 0.3415 - accuracy: 0.8688
Epoch 3/100
2507/2507 [==============================] - 2s 856us/step - loss: 0.2160 - accuracy: 0.8975
Epoch 4/100
2507/2507 [==============================] - 2s 855us/step - loss: 0.1575 - accuracy: 0.9194
Epoch 5/100
2507/2507 [==============================] - 2s 836us/step - loss: 0.1246 - accuracy: 0.9282
Epoch 6/100
2507/2507 [==============================] - 2s 834us/step - loss: 0.1153 - accuracy: 0.9238
Epoch 7/100
2507/2507 [==============================] - 2s 860us/step - loss: 0.1069 - accuracy: 0.9230
Epoch 8/100
2507/2507 [==============================] - 2s 839us/step - loss: 0.1031 - accuracy: 0.9350
Epoch 9/100
2507/2507 [==============================] - 2s 827us/step - loss: 0.1025 - accuracy: 0.9314
Epoch 10/100
2507/2507 [==============================]

2507/2507 [==============================] - 2s 831us/step - loss: 0.0944 - accuracy: 0.9334
Epoch 79/100
2507/2507 [==============================] - 2s 815us/step - loss: 0.0946 - accuracy: 0.9394
Epoch 80/100
2507/2507 [==============================] - 2s 829us/step - loss: 0.0940 - accuracy: 0.9358
Epoch 81/100
2507/2507 [==============================] - 2s 827us/step - loss: 0.0941 - accuracy: 0.9374
Epoch 82/100
2507/2507 [==============================] - 2s 825us/step - loss: 0.0945 - accuracy: 0.9350
Epoch 83/100
2507/2507 [==============================] - 2s 812us/step - loss: 0.0948 - accuracy: 0.9366
Epoch 84/100
2507/2507 [==============================] - 2s 819us/step - loss: 0.0948 - accuracy: 0.9374
Epoch 85/100
2507/2507 [==============================] - 2s 825us/step - loss: 0.0976 - accuracy: 0.9350
Epoch 86/100
2507/2507 [==============================] - 2s 829us/step - loss: 0.0949 - accuracy: 0.9342
Epoch 87/100
2507/2507 [==============================] - 2

In [12]:

test_reviews = [
    "The food was lousy - too sweet or too salty and the portions tiny",
    "After all that, they complained to me about the small tip",
    "I have eaten at Saul, many times, the food is always consistently, outrageously good",
    "The duck confit is always amazing and the foie gras terrine with figs was out of this world",
    "Avoid this place!"
]
                             
# Sentiment preprocessing
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 
test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))

# Models output
test_sentiment = label_encoder.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))


In [13]:
for i,j in zip(test_reviews,test_sentiment):
    print("Review:"+" "+str(i)+" "+"is expressing a"+ " " +str(j))

Review: The food was lousy - too sweet or too salty and the portions tiny is expressing a negative
Review: After all that, they complained to me about the small tip is expressing a positive
Review: I have eaten at Saul, many times, the food is always consistently, outrageously good is expressing a positive
Review: The duck confit is always amazing and the foie gras terrine with figs was out of this world is expressing a positive
Review: Avoid this place! is expressing a negative
